### 1.1 Gather the title and genre of the maximum top 10 movies that each user clicked on regarding the number of clicks.

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import pandas as pd
import numpy as np
import time
import itertools
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
dataset = pd.read_csv("vodclickstream_uk_movies_03.csv")
dataset.head()

,Unnamed: 0,datetime,duration,title,genres,release_date,movie_id,user_id
0,58773,2017-01-01 01:15:09,0.0,"Angus, Thongs and Perfect Snogging","Comedy, Drama, Romance",2008-07-25,26bd5987e8,1dea19f6fe
1,58774,2017-01-01 13:56:02,0.0,The Curse of Sleeping Beauty,"Fantasy, Horror, Mystery, Thriller",2016-06-02,f26ed2675e,544dcbc510
2,58775,2017-01-01 15:17:47,10530.0,London Has Fallen,"Action, Thriller",2016-03-04,f77e500e7a,7cbcc791bf
3,58776,2017-01-01 16:04:13,49.0,Vendetta,"Action, Drama",2015-06-12,c74aec7673,ebf43c36b6
4,58777,2017-01-01 19:16:37,0.0,The SpongeBob SquarePants Movie,"Animation, Action, Adventure, Comedy, Family, ...",2004-11-19,a80d6fc2aa,a57c992287


After reading the data, I will change the data type for the following columns: [datetime and release_date] to a pandas date time object. Also, I will check how many null values are present in my dataset. Lastly, I will replace the name of the column "Unnamed: 0" with "session", as the name makes more sense. 

In [3]:
dataset.datetime = pd.to_datetime(dataset.datetime)
dataset.release_date = pd.to_datetime(dataset.release_date, errors='coerce')
print(dataset.isnull().sum())
dataset.rename(columns={'Unnamed: 0': 'session'}, inplace=True)

Unnamed: 0          0
datetime            0
duration            0
title               0
genres              0
release_date    30304
movie_id            0
user_id             0
dtype: int64


We have 30304 missing values for release_date column. But for our purpose, this won't be an issue.

Now, we have to gather the **title and genre** of the **top 10** movies, with regards to the **number of clicks** per user. To be able to break down the dataset into a form where I can access the top ten movie titles and genres for each user, I will use `groupby()` and `sort_values()` repetitively on the dataframe.

In [4]:
user_click_counts = dataset.groupby(['user_id', 'title', 'genres']).size().reset_index(name='click_count')

# Sort the movies for each user based on click counts
user_click_counts = user_click_counts.sort_values(['user_id', 'click_count'], ascending=[True, False])

# Get the top 10 movies for each user
top_10_movies_per_user = user_click_counts.groupby('user_id').head(10)

# Extract title and genre of the top 10 movies
df = top_10_movies_per_user[['user_id', 'title', 'genres', 'click_count']]

In [5]:
df.head()

,user_id,title,genres,click_count
0,00004e2862,Hannibal,"Crime, Drama, Thriller",1
6,000052a0a0,Looper,"Action, Drama, Sci-Fi, Thriller",9
3,000052a0a0,Frailty,"Crime, Drama, Thriller",3
5,000052a0a0,Jumanji,"Adventure, Comedy, Family, Fantasy",3
7,000052a0a0,Resident Evil,"Action, Horror, Sci-Fi",2


### 1.2

#### Using the movie genre and user_ids, try to implement your min-hash signatures so that users with similar interests in a genre appear in the same bucket.

---

##### Important note: You must write your minhash function from scratch. You are not permitted to use any already implemented hash functions. Read the class materials and, if necessary, conduct an internet search. The description of hash functions in the book may be helpful as a reference.

---
Now, I will define a funtion, name as `get_unique_genres_for_user()`, which will aid in storing **storing** unique genres for each user in the `df` dataframe. I will, again, `groupby()` the `df` dataframe by `user_id` column, select the `genres` column and apply our `get_unique_genres_for_user()` function across the `genres` column. This will output a dataframe, that I have named `user_genres`, which contains two columns: `user_id` and `genres`. `user_id` is a column with, as you will see, unique users and `genres` is a column which has a string of unique genres - concatenated together by commas. 

In [6]:
def get_unique_genres_for_user(group):
    # Combine all genres for a user into a single list
    all_genres = [genre.split(', ') for genre in group]
    # Flatten the list of lists and create a set of unique genres
    unique_genres = set([item for sublist in all_genres for item in sublist])
    return ', '.join(unique_genres)

In [7]:
# grouping on user_id, then selection the genres column and applying our function!
user_genres = df.groupby('user_id')['genres'].apply(lambda x: get_unique_genres_for_user(x)).reset_index()
print(user_genres.shape)
user_genres.head(3)

(161918, 2)


,user_id,genres
0,00004e2862,"Crime, Drama, Thriller"
1,000052a0a0,"Action, Comedy, Mystery, Thriller, Horror, Adv..."
2,000090e7c8,"Mystery, Thriller, Sci-Fi"


I will now check to verify if all the values are unique:

In [8]:
user_genres.user_id.nunique()

161918

Indeed the number of unique user_ids matches the size of the dataframe. Lastly, I will just explore one random entry of the genres column.

In [9]:
user_genres.genres[1]

'Action, Comedy, Mystery, Thriller, Horror, Adventure, Fantasy, Family, Sci-Fi, Crime, Drama, Sport, Music'

First stop in our Locality Sensitive Hashing process is: **k-Shingling**, or simply **Shingling**. It is the process of converting a string of text into a set of ‘shingles’. The process is similar to moving a window of length k down our string of text and taking a picture at each step. We collate all of those pictures to create our set of shingles. But for our purpose, we will intake the complete word from the genre column.

In [10]:
# Convert genre string to list ~ Shingle for each user.
user_genres['genres'] = user_genres['genres'].apply(lambda x: x.split(', '))

# Get all unique genres
unique_genres = set()
for genres in user_genres['genres']:
    unique_genres.update(genres)

unique_genres = list(unique_genres)
unique_genres.sort()

# Create a dictionary to map genres to column indices - it will help in one hot encoding later.
genre_to_index = {genre: i for i, genre in enumerate(unique_genres)}

Now, I will build a sparse (one hot encoding matrix), also know as characteristics matrix for our data. Rows are the elements (shingles) and columns will be the User_ids in which they appeared. The entry will be 1 if the element (shingle) is in the column (user_id).

I will follow the procedure of the book.

<img src = "https://drive.google.com/uc?id=1n752VizpcQaPKajZNjjfuQHd9MQy5e0W">

In [11]:
# Create a matrix representation (one-hot encoding) for users' genres
matrix_representation = [] # matrix initialisation 
for genres in user_genres['genres']: # for each genre_list in user_genres['genres'] column
    # Create a row filled with zeros equal to the number of unique genres
    row = [0] * len(unique_genres)
    for genre in genres: # for each genre in the genres (list)
        # Set the index in the row to 1 for genre index is present in genre_to_index dict
        row[genre_to_index[genre]] = 1
    matrix_representation.append(row)

matrix_df = pd.DataFrame(matrix_representation, columns=unique_genres) # initialisation of empty df
matrix_df.set_index(user_genres['user_id'], inplace=True) # setting index to the user_id column of our original dataframe
matrix_df.index.name = None # to get rid of name when transpose will be applied
matrix_df = matrix_df.T # transposing to get the format, as in the book
print(matrix_df.shape)
matrix_df.iloc[:,:10] # all rows (genres) and first 10 columns.

(27, 161918)


,00004e2862,000052a0a0,000090e7c8,000118a755,000296842d,0002aab109,0002abf14f,0002d1c4b1,000499c2b6,00051f0e1f
Action,0,1,0,0,0,0,0,0,0,1
Adventure,0,1,0,0,0,0,0,0,1,1
Animation,0,0,0,0,0,0,0,0,1,0
Biography,0,0,0,0,0,1,0,0,0,0
Comedy,0,1,0,0,0,1,0,1,1,1
Crime,1,1,0,0,0,1,0,0,0,0
Documentary,0,0,0,0,0,0,0,0,0,0
Drama,1,1,0,0,1,1,1,0,0,0
Family,0,1,0,0,0,0,0,0,1,0
Fantasy,0,1,0,0,0,0,0,0,1,0


Minhashing is the next step in our LSH process, allowing us to convert our sparse vectors into dense vectors. But before we can perform minhashing on our sparse vectors, what we do is generate some carefully chosen hash functions that simulate the effect of permuting the entire rows uniquely. Also, the number of hash functions we chose here would reflect the number of rows we want to create in our dense vector/signature. For example, If we want 8 rows in our signature matrix (dense representation), we would use 8 hash functions. At the end of this, we produce our minhash signature — or dense vector.

Again, I will be following the algorith provided by the author of the book:

---
<img src = "https://drive.google.com/uc?id=1P48autheVmPblZ3k2RrJ97cS_URD0Thu" width="600" height="100">

In [12]:
# Define the number of: hash functions, unique genres, and hash functions with specific 'a' and 'b' values
num_hash_functions = 8
num_genres = len(unique_genres)
# the values of a and b here are chosen carefully, after much trial and error. they simulate a true permuation.
hash_functions = [(82, 15), (4, 95), (1, 3), (29, 18), (95, 14), (7, 7), (70, 12), (76, 55)]

# Convert the DataFrame to a NumPy array for faster computations
matrix_array = matrix_df.values.astype(int)

# Initialize the signature matrix with infinity values
# of dimension num_hash_functions * num_of_users
signature_matrix = np.full((num_hash_functions, matrix_array.shape[1]), np.inf) 

for r in range(matrix_array.shape[0]):  # Iterate through rows
    for c in range(matrix_array.shape[1]):  # Iterate through columns
        if matrix_array[r, c] != 0: # for all non zero value
            for i, (a, b) in enumerate(hash_functions):  # Iterate through hash functions
                hash_val = ((a * r + b) % num_genres)  # Calculate the hash value
                signature_matrix[i, c] = min(signature_matrix[i, c], hash_val) # take the min and plug it in sig matrix

In [13]:
signature_df = pd.DataFrame(data=signature_matrix, columns=matrix_df.columns)
signature_df

,00004e2862,000052a0a0,000090e7c8,000118a755,000296842d,0002aab109,0002abf14f,0002d1c4b1,000499c2b6,00051f0e1f,...,fffb9ecb47,fffc1d209b,fffd345213,fffd4d1888,fffd6433d2,fffd9bf758,fffe7b777b,fffeac83be,ffff2c5f9e,ffffd36adf
0,12.0,0.0,3.0,0.0,3.0,18.0,7.0,8.0,16.0,8.0,...,3.0,8.0,16.0,7.0,6.0,3.0,8.0,3.0,3.0,20.0
1,2.0,2.0,2.0,8.0,2.0,3.0,9.0,3.0,3.0,3.0,...,3.0,3.0,3.0,2.0,5.0,7.0,2.0,3.0,2.0,7.0
2,0.0,0.0,0.0,15.0,0.0,6.0,10.0,7.0,4.0,3.0,...,6.0,2.0,4.0,0.0,21.0,8.0,0.0,1.0,0.0,8.0
3,1.0,1.0,4.0,15.0,4.0,1.0,2.0,4.0,7.0,4.0,...,1.0,1.0,7.0,2.0,0.0,1.0,4.0,1.0,3.0,1.0
4,3.0,1.0,8.0,20.0,4.0,2.0,4.0,16.0,1.0,1.0,...,2.0,0.0,1.0,4.0,23.0,3.0,4.0,2.0,8.0,3.0
5,2.0,2.0,4.0,10.0,2.0,1.0,2.0,8.0,8.0,7.0,...,1.0,0.0,8.0,2.0,25.0,4.0,2.0,1.0,4.0,2.0
6,11.0,1.0,8.0,15.0,8.0,6.0,16.0,8.0,1.0,1.0,...,0.0,1.0,1.0,12.0,3.0,0.0,8.0,0.0,0.0,11.0
7,3.0,1.0,7.0,2.0,7.0,3.0,14.0,8.0,8.0,1.0,...,3.0,1.0,8.0,1.0,19.0,3.0,9.0,0.0,1.0,3.0


As can be analysed from the result, we have reduced the dimension of the rows from **27*161918** to **8*161918**. But one might ask, why even minhash? Would the similarity perspective be preserved?

Surpirsingly, **YES**. There is a remarkable connection between minhashing and Jaccard similarity of the sets that are minhashed. The probability that the minhash function for a random permutation of rows produces the same value for two sets equals the Jaccard similarity of those sets.

---

The main idea behind LSH is to efficiently group similar items by hashing them multiple times to the same bucket, aiming to minimize dissimilar items ending up in the same bucket. By considering pairs that hash to the same bucket across various hashings as candidates for similarity comparison, LSH reduces computational load. 

The goal is to optimize true similar pairs hashing to the same bucket while limiting false positives. When using minhash signatures, LSH divides the signature matrix into **bands**. Each **band applies a hash function to vectors of integers, directing them to numerous buckets**. In other words, we want to maximize collisions — although ideally only for similar inputs.

The number of bands must follow this equation: n = b x r. Here, n is the total number of rows, which in our case is 8, b is the number of bands one would like to use, and r would be the number of rows within each band. Let's deploy this concept.

In [34]:
from collections import defaultdict
#4 bands, 2 rows = 8 rows (total)
num_bands = 4  # Define the number of bands
rows_per_band = signature_matrix.shape[0] // num_bands  # Define the number of rows per band
n_users = signature_matrix.shape[1]  # Number of users/customers

# Initialize a dictionary to store buckets
buckets = defaultdict(list)

# Define a Hashing function based on our minhash value
def minhash_based_hashing(signature_values):
    # Convert the signature values to a string for hashing
    signature_sum = sum(signature_values)
    return ((23*signature_sum + 89) % 997) # return the string HASHED


# Perform LSH by hashing users into buckets
for idx_user, column in enumerate(signature_matrix.T):  # Iterate over columns (users)
    for idx_band, band in enumerate(np.split(column, num_bands)):  # Split into bands
        # Create keys for the buckets using a tuple of band index and band values
        bucket_key = minhash_based_hashing(band.tolist())
        buckets[bucket_key].append(idx_user)  # Append the user index to the corresponding bucket

In [35]:
len(buckets) # we have 354 distinct buckets!!!

# our bucket is a dict defined in a way that the key is the value of the hash function and value is the list 
# of user_ids that go (hash into) these buckets

46

### 1.3 Locality-Sensitive Hashing (LSH)

Now that your buckets are ready, it's time to ask a few queries. We will provide you with some user_ids and ask you to recommend at most five movies to the user to watch based on the movies clicked by similar users.

To recommend at most five movies given a user_id, use the following procedure:

1. Identify the two most similar users to this user.
2. If these two users have any movies in common, recommend those movies based on the total number of clicks by these users.
3. If there are no more common movies, try to propose the most clicked movies by the most similar user first, followed by the other user.

**Example:** assume you've identified user **A** and **B** as the most similar users to a single user, and we have the following records on these users:

- User A with 80% similarity
- User B with 50% similarity

| user | movie title | #clicks |
|----------|----------|----------|
| A | Wild Child | 20 |
| A | Innocence | 10 |
| A | Coin Heist | 2 |
| B | Innocence | 30 |
| B | Coin Heist | 15 |
| B | Before I Fall | 30 |
| B | Beyond Skyline | 8 |
| B | The Amazing Spider-Man | 5 |

**Recommended movies** in order:
- Innocence
- Coin Heist
- Wild Child
- Before I Fall
- Beyond Skyline

---

In [36]:
random_col = random.randint(1, signature_df.shape[1]) # a random column number
query = signature_df.iloc[:,random_col].name # to get the name of the col (user_id)
user_id_index = signature_df.columns.get_loc(query) # get the index of the user_id col
user_id_index

1901

In [38]:
def jaccard_similarity(set1, set2):
    # Calculate Jaccard similarity between two sets
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

def find_two_most_similar_users(user_id, buckets):
    # Find the bucket containing the given user
    user_buckets = set() # I assume there is a chance that a user is present in more than one bucket, tho it is rare.
    for bucket_key, users in buckets.items():
        if user_id in users: # if our user (the query) is in the list
            user_buckets.update(users) # update the user_buckets with the list of similar users
        
    if not user_buckets:
        return None  # User not found in any bucket

    # Calculate similarity of the given user with users in the bucket
    similarities = {}
# calculate the jaccard_similarity between query and every other user in our bucket/list of similar users
    for other_user_id in user_buckets:
        if other_user_id != user_id:
            user_set = set(signature_matrix[:, user_id])
            other_user_set = set(signature_matrix[:, other_user_id])
            similarities[other_user_id] = jaccard_similarity(user_set, other_user_set)

    # Find the two most similar users
    most_similar_users = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:2]
    return most_similar_users


In [39]:
most_similar_users = find_two_most_similar_users(user_id_index, buckets)
print(most_similar_users)

[(131144, 1.0), (131333, 1.0)]


How to interpret the result? the first value in the tuple is the user_id and the second value is the jaccard similarity

In [40]:
list_of_users = [user for user, _ in most_similar_users]
list_of_scores = [score for _, score in most_similar_users]
user_ids = list()
for user in list_of_users:
    user_ids.append(signature_df.iloc[:,user].name)
user_ids # this will help in making the movies dataframe of these users

['cf644231f2', 'cfb34b3a77']

In [41]:
movies = df[df["user_id"].isin(user_ids)].reset_index(drop=True)
movies

,user_id,title,genres,click_count
0,cf644231f2,The Ballad of Buster Scruggs,"Comedy, Drama, Musical, Mystery, Romance, Western",1
1,cfb34b3a77,The Ballad of Buster Scruggs,"Comedy, Drama, Musical, Mystery, Romance, Western",1


If these two users have any movies in common, recommend those movies based on the total number of clicks by these users.

In [42]:
two_similar_users = list()
for x,y in zip(user_ids, list_of_scores):
    two_similar_users.append((x,y))
two_similar_users # simialr to most_similar_users, but with the original user_id and not the col_index

[('cf644231f2', 1.0), ('cfb34b3a77', 1.0)]

In [43]:
def recommend_movies(movies, two_similar_users):
    """
    Given data of movies, I have to do the following:
    initialise a list called: results
    First, check if there are some common titles between two users:
    for each common title, count the number of clicks by each user and sum it. then, sort the common titles by the sum of the number of clicks by each user and append the name of the title into the list: results.
    Then, If there are no more common movies, try to propose the most clicked movies by the most similar user first, followed by the other user. add those result after the results already in the list: results.
    then print the first 5 results of the list: result
    """
    # Find common movies
    user_ids = [user[0] for user in two_similar_users]
    common_movies = set(movies[movies['user_id'] == user_ids[0]]['title']).intersection(
        set(movies[movies['user_id'] == user_ids[1]]['title'])
    )

    # Initialize a set called 'results' to avoid duplicates
    results = list()
    common_movies_df = pd.DataFrame
    if common_movies:
        # If there are common movies, recommend those based on the total number of clicks by these users
        common_movies_df = movies[movies['title'].isin(common_movies)].groupby('title')['click_count'].sum().reset_index()
        common_movies_df = common_movies_df.sort_values(by='click_count', ascending=False)
        results.extend(common_movies_df['title'].tolist())

    # Propose the most clicked movies by each user
    if not common_movies_df.empty:
        movies = movies[~movies["title"].isin(common_movies_df['title'].tolist())]
    for user_id, _ in two_similar_users: # first user_id is automatically with the largest similarity score
        user_movies = movies[movies['user_id'] == user_id].groupby('title')['click_count'].sum().reset_index()
        user_movies = user_movies.sort_values(by='click_count', ascending=False)
        results.extend(user_movies['title'].tolist())

    # Return the recommended movie titles
    return list(results)[:5]  # Return the first 5 unique results

In [44]:
recommend_movies(movies, two_similar_users)

['The Ballad of Buster Scruggs']

Credits: `https://www.pinecone.io/learn/series/faiss/locality-sensitive-hashing/`and Mining of Massive Datasets Book.